In [1]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
__author__ = 'Author'
__email__ = 'Email'

# SemEval 2026 Task 5 - Ensemble

In [2]:
# dependency
# built-in
import json
import os
import sys
from pathlib import Path
from itertools import combinations

# third-party
import random
import pandas as pd
import numpy as np
from scipy import stats
from xgboost import XGBRanker, XGBRegressor
from scipy.stats import spearmanr
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# local - add src/eval to path for importing evaluation functions
sys.path.insert(0, str(Path('../src/eval').resolve()))
# Import evaluation functions from src/eval/scoring.py
import scoring

%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Importing...
Starting Scoring script...


# Init

In [3]:
# helper
def set_seed(seed=42):
    """Set random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)

def load_predictions(filepath):
    """Load predictions from a JSONL file into a dictionary."""
    predictions = {}
    with open(filepath, 'r') as f:
        for line in f:
            data = json.loads(line.strip())
            predictions[data['id']] = data['prediction']
    return predictions

def convert_json_to_submittable_jsonl(input_json_path, output_jsonl_path):
    """
    Convert from detailed JSON format to submittable JSONL format.
    
    Input format (JSON array):
        [{"id": "0", "homonym": "...", "model_score": 5, ...}, ...]
    
    Output format (JSONL):
        {"id": "0", "prediction": 5}
        {"id": "1", "prediction": 2}
        ...
    
    Args:
        input_json_path: Path to input JSON file (e.g., chatgpt_v1.json)
        output_jsonl_path: Path to output JSONL file (e.g., chatgpt_v1_submission.jsonl)
    """
    # Read the JSON array
    with open(input_json_path, 'r') as f:
        data = json.load(f)

    # Write to JSONL format
    with open(output_jsonl_path, 'w') as f:
        for item in data:
            json_obj = {
                "id": item["id"],
                "prediction": int(item["model_score"])
            }
            f.write(json.dumps(json_obj) + '\n')

    print(f"Converted {len(data)} predictions")
    print(f"Saved to: {output_jsonl_path}")

# Evaluation functions following the structure of src/eval/scoring.py
def evaluate_predictions_array(y_pred, y_true_labels):
    """
    Evaluate predictions using the same logic as scoring.py
    
    Args:
        y_pred: array of predictions
        y_true_labels: list of gold label lists (5 ratings each)
    
    Returns:
        dict with spearman and accuracy scores
    """
    # Build prediction and gold lists (same structure as scoring.py)
    pred_list = list(y_pred)
    gold_list = [scoring.get_average(labels) for labels in y_true_labels]
    
    # Calculate Spearman correlation (same as scoring.py)
    corr, p_value = spearmanr(pred_list, gold_list)
    
    # Calculate accuracy within SD (same logic as scoring.py)
    correct_guesses = 0
    wrong_guesses = 0
    
    for pred, labels in zip(pred_list, y_true_labels):
        if scoring.is_within_standard_deviation(pred, labels):
            correct_guesses += 1
        else:
            wrong_guesses += 1
    
    accuracy = correct_guesses / (correct_guesses + wrong_guesses)
    
    return {
        'spearman': corr,
        'p_value': p_value,
        'accuracy': accuracy,
        'correct': correct_guesses,
        'total': correct_guesses + wrong_guesses
    }

In [4]:
# init
set_seed(0)

## Data

In [5]:
# import glob
# from pathlib import Path

# base_path = '../res/results/test/*.json'

# json_files = glob.glob(base_path)
# for json_file in json_files:
#     json_path = Path(json_file)
#     output_path = json_file.replace('json', 'jsonl')
#     convert_json_to_submittable_jsonl(json_file, output_path)

In [6]:

# Load gold labels (solution file)
SOLUTION_FILE = Path("../res/data/dev_solution.jsonl")

gold_labels = {}
with open(SOLUTION_FILE, 'r') as f:
    for line in f:
        data = json.loads(line.strip())
        gold_labels[data['id']] = data['label']

print(f"Loaded {len(gold_labels)} gold labels")
print(f"\nGold labels are lists of 5 human ratings (1-5 scale)")
print(f"\nExample gold labels:")
for i in range(5):
    sample_id = str(i)
    print(f"  ID {sample_id}: {gold_labels[sample_id]} (avg={np.mean(gold_labels[sample_id]):.2f}, std={np.std(gold_labels[sample_id], ddof=1):.2f})")

Loaded 588 gold labels

Gold labels are lists of 5 human ratings (1-5 scale)

Example gold labels:
  ID 0: [4, 5, 3, 1, 5] (avg=3.60, std=1.67)
  ID 1: [3, 3, 4, 4, 4] (avg=3.60, std=0.55)
  ID 2: [5, 5, 2, 3, 4] (avg=3.80, std=1.30)
  ID 3: [4, 5, 4, 3, 5] (avg=4.20, std=0.84)
  ID 4: [1, 5, 4, 4, 1] (avg=3.00, std=1.87)


## System

In [7]:
# Path to individual system outputs
RESULTS_DIR = Path("./dev/")

# Get all jsonl files
prediction_files = sorted(RESULTS_DIR.glob("*.jsonl"))
print(f"Found {len(prediction_files)} prediction files:")
for f in prediction_files:
    print(f"  - {f.name}")

Found 18 prediction files:
  - chatgpt_v1.jsonl
  - david_v1.jsonl
  - david_v2.jsonl
  - david_v3.jsonl
  - deepseek_v1.jsonl
  - gemini.jsonl
  - gpt5_v1.jsonl
  - gpt5_v3.jsonl
  - gpt5_v4.jsonl
  - korean_v1.jsonl
  - korean_v2.jsonl
  - majoritybaseline.jsonl
  - qwen_v1.jsonl
  - qwen_v2.jsonl
  - randombaseline.jsonl
  - urdu_v1.jsonl
  - urdu_v2.jsonl
  - urdu_v3.jsonl


In [8]:
# Load all predictions
all_predictions = {}
for pred_file in prediction_files:
    system_name = pred_file.stem  # filename without extension
    all_predictions[system_name] = load_predictions(pred_file)
    print(f"Loaded {len(all_predictions[system_name])} predictions from {system_name}")

# Convert to DataFrame for easier manipulation
# Each row is a sample, each column is a system's prediction
df_predictions = pd.DataFrame(all_predictions)
df_predictions.index.name = 'id'

print(f"\nPredictions DataFrame shape: {df_predictions.shape}")
print(f"Number of samples: {len(df_predictions)}")
print(f"Number of systems: {len(df_predictions.columns)}")
print(f"\nSystems: {list(df_predictions.columns)}")
print(f"\nSample data types:")
print(df_predictions.dtypes)
print(f"\nFirst 10 predictions:")
df_predictions.head(10)

Loaded 588 predictions from chatgpt_v1
Loaded 588 predictions from david_v1
Loaded 588 predictions from david_v2
Loaded 588 predictions from david_v3
Loaded 588 predictions from deepseek_v1
Loaded 588 predictions from gemini
Loaded 588 predictions from gpt5_v1
Loaded 588 predictions from gpt5_v3
Loaded 588 predictions from gpt5_v4
Loaded 588 predictions from korean_v1
Loaded 588 predictions from korean_v2
Loaded 588 predictions from majoritybaseline
Loaded 588 predictions from qwen_v1
Loaded 588 predictions from qwen_v2
Loaded 588 predictions from randombaseline
Loaded 588 predictions from urdu_v1
Loaded 588 predictions from urdu_v2
Loaded 588 predictions from urdu_v3

Predictions DataFrame shape: (588, 18)
Number of samples: 588
Number of systems: 18

Systems: ['chatgpt_v1', 'david_v1', 'david_v2', 'david_v3', 'deepseek_v1', 'gemini', 'gpt5_v1', 'gpt5_v3', 'gpt5_v4', 'korean_v1', 'korean_v2', 'majoritybaseline', 'qwen_v1', 'qwen_v2', 'randombaseline', 'urdu_v1', 'urdu_v2', 'urdu_v3']


,chatgpt_v1,david_v1,david_v2,david_v3,deepseek_v1,gemini,gpt5_v1,gpt5_v3,gpt5_v4,korean_v1,korean_v2,majoritybaseline,qwen_v1,qwen_v2,randombaseline,urdu_v1,urdu_v2,urdu_v3
id,,,,,,,,,,,,,,,,,,
0,5,4,4.0600,1,4,5,5,4,5,4.360252,4.530886,4,5,4,4,4,4,4
1,2,4,1.9400,5,2,2,2,1,1,2.586742,2.275880,4,1,2,3,3,2,3
2,3,5,3.9976,1,3,4,4,2,2,2.647276,3.408030,4,3,3,5,4,3,4
3,2,5,2.0024,5,3,5,4,4,4,3.392001,3.437395,4,3,3,5,4,4,4
4,3,5,3.5480,5,3,4,3,4,4,2.816402,3.001934,4,5,3,2,4,4,4
5,2,4,2.4520,5,3,5,4,4,4,3.373284,3.141558,4,2,3,4,4,3,4
6,5,4,4.5992,1,4,5,5,4,5,4.360252,4.341353,4,1,4,2,2,4,2
7,1,1,1.4008,1,2,1,1,1,1,1.942182,1.744660,4,1,2,1,2,3,2
8,1,3,2.8696,1,4,2,1,1,1,1.942182,2.118062,4,5,4,2,2,2,2


In [9]:
# Create a combined DataFrame with predictions and gold labels
df_gold = pd.DataFrame({
    'gold_labels': gold_labels,
    'gold_avg': {k: np.mean(v) for k, v in gold_labels.items()},
    'gold_std': {k: np.std(v, ddof=1) for k, v in gold_labels.items()}
})

# Combine predictions with gold labels
df_combined = df_predictions.join(df_gold)

print(f"Combined DataFrame shape: {df_combined.shape}")
print(f"\nColumns: {list(df_combined.columns)}")
print(f"\nFirst 10 rows:")
df_combined.head(10)

Combined DataFrame shape: (588, 21)

Columns: ['chatgpt_v1', 'david_v1', 'david_v2', 'david_v3', 'deepseek_v1', 'gemini', 'gpt5_v1', 'gpt5_v3', 'gpt5_v4', 'korean_v1', 'korean_v2', 'majoritybaseline', 'qwen_v1', 'qwen_v2', 'randombaseline', 'urdu_v1', 'urdu_v2', 'urdu_v3', 'gold_labels', 'gold_avg', 'gold_std']

First 10 rows:


,chatgpt_v1,david_v1,david_v2,david_v3,deepseek_v1,gemini,gpt5_v1,gpt5_v3,gpt5_v4,korean_v1,...,majoritybaseline,qwen_v1,qwen_v2,randombaseline,urdu_v1,urdu_v2,urdu_v3,gold_labels,gold_avg,gold_std
id,,,,,,,,,,,,,,,,,,,,,
0,5,4,4.0600,1,4,5,5,4,5,4.360252,...,4,5,4,4,4,4,4,"[4, 5, 3, 1, 5]",3.600000,1.673320
1,2,4,1.9400,5,2,2,2,1,1,2.586742,...,4,1,2,3,3,2,3,"[3, 3, 4, 4, 4]",3.600000,0.547723
2,3,5,3.9976,1,3,4,4,2,2,2.647276,...,4,3,3,5,4,3,4,"[5, 5, 2, 3, 4]",3.800000,1.303840
3,2,5,2.0024,5,3,5,4,4,4,3.392001,...,4,3,3,5,4,4,4,"[4, 5, 4, 3, 5]",4.200000,0.836660
4,3,5,3.5480,5,3,4,3,4,4,2.816402,...,4,5,3,2,4,4,4,"[1, 5, 4, 4, 1]",3.000000,1.870829
5,2,4,2.4520,5,3,5,4,4,4,3.373284,...,4,2,3,4,4,3,4,"[4, 3, 4, 1, 3]",3.000000,1.224745
6,5,4,4.5992,1,4,5,5,4,5,4.360252,...,4,1,4,2,2,4,2,"[4, 4, 5, 5, 5]",4.600000,0.547723
7,1,1,1.4008,1,2,1,1,1,1,1.942182,...,4,1,2,1,2,3,2,"[1, 1, 1, 2, 2, 1]",1.333333,0.516398
8,1,3,2.8696,1,4,2,1,1,1,1.942182,...,4,5,4,2,2,2,2,"[4, 1, 1, 2, 3]",2.200000,1.303840


# Split Data

In [10]:
# Use stratification based on binned gold_avg to ensure balanced distribution
df_combined['gold_bin'] = pd.cut(df_combined['gold_avg'], bins=5, labels=False)

train_df, test_df = train_test_split(
    df_combined, 
    test_size=0.3, 
    random_state=42,
    stratify=df_combined['gold_bin']
)

# Drop the temporary binning column
train_df = train_df.drop('gold_bin', axis=1)
test_df = test_df.drop('gold_bin', axis=1)

train_df.to_csv('our_dev_splits/train.tsv', sep='\t')
test_df.to_csv('our_dev_splits/test.tsv', sep='\t')

test_gold_labels = test_df['gold_labels'].tolist()

print(f"Total samples: {len(df_combined)}")
print(f"Training samples: {len(train_df)} ({len(train_df)/len(df_combined)*100:.1f}%)")
print(f"Test samples: {len(test_df)} ({len(test_df)/len(df_combined)*100:.1f}%)")

print(f"\nTraining set gold_avg distribution:")
print(f"  Mean: {train_df['gold_avg'].mean():.3f}")
print(f"  Std: {train_df['gold_avg'].std():.3f}")
print(f"  Min: {train_df['gold_avg'].min():.3f}")
print(f"  Max: {train_df['gold_avg'].max():.3f}")

print(f"\nTest set gold_avg distribution:")
print(f"  Mean: {test_df['gold_avg'].mean():.3f}")
print(f"  Std: {test_df['gold_avg'].std():.3f}")
print(f"  Min: {test_df['gold_avg'].min():.3f}")
print(f"  Max: {test_df['gold_avg'].max():.3f}")

print(f"\nTraining set sample IDs range: {train_df.index.min()} to {train_df.index.max()}")
print(f"Test set sample IDs range: {test_df.index.min()} to {test_df.index.max()}")

Total samples: 588
Training samples: 411 (69.9%)
Test samples: 177 (30.1%)

Training set gold_avg distribution:
  Mean: 3.109
  Std: 1.191
  Min: 1.000
  Max: 5.000

Test set gold_avg distribution:
  Mean: 3.140
  Std: 1.178
  Min: 1.000
  Max: 5.000

Training set sample IDs range: 0 to 97
Test set sample IDs range: 1 to 99


In [11]:
def prepare_data_splits(train_df, test_df, model_names):
    """
    Prepare training and test sets with specified models.

    Args:
        train_df: Training dataframe with all model predictions
        test_df: Test dataframe with all model predictions
        model_names: List of model column names to use as features

    Returns:
        X_train, y_train, X_test, y_test
    """
    X_train = train_df[model_names]
    y_train = train_df['gold_avg']

    X_test = test_df[model_names]
    y_test = test_df['gold_avg']

    # print(f"X_train shape: {X_train.shape}")
    # print(f"y_train shape: {y_train.shape}")
    # print(f"X_test shape: {X_test.shape}")
    # print(f"y_test shape: {y_test.shape}")

    # print(f"\nSelected models: {model_names}")
    # print(f"\nFirst few training samples:")
    # display(pd.concat([X_train.head(), y_train.head()], axis=1))

    return X_train, y_train, X_test, y_test

# System Evaluation

In [12]:
# Evaluate each system on test set
print("=" * 80)
print("Individual System Performance on Test Set (30%)")
print("=" * 80)
print(f"{'System':<15} {'Spearman':>12} {'Accuracy':>12} {'Correct/Total':>15}")
print("-" * 80)

system_results = {}
test_gold_labels = test_df['gold_labels'].tolist()


all_systems = [s for s in list(df_combined.columns) if not s.startswith('gold_')]
X_train, y_train, X_test, y_test = prepare_data_splits(train_df, test_df, all_systems)

for system in all_systems:
# for system in ['david_v1', 'david_v2', 'korean']:
    # Get predictions from X_test
    y_pred = X_test[system].values
    
    # Evaluate using official logic
    scores = evaluate_predictions_array(y_pred, test_gold_labels)
    system_results[system] = scores
    
# sort systems by average of spearman and accuracy
sorted_systems = sorted(system_results.items(), key=lambda x: (x[1]['spearman'] + x[1]['accuracy'])/2, reverse=True)
for system, scores in sorted_systems:
    print(f"{system:<15} {scores['spearman']:>12.8f} {scores['accuracy']:>12.8f} "
          f"{scores['correct']:>7}/{scores['total']:<7}")
print("=" * 80)

Individual System Performance on Test Set (30%)
System              Spearman     Accuracy   Correct/Total
--------------------------------------------------------------------------------
korean_v1         0.82703251   0.90395480     160/177    
korean_v2         0.83099820   0.89830508     159/177    
gpt5_v1           0.77775958   0.76271186     135/177    
gpt5_v4           0.74834361   0.73446328     130/177    
majoritybaseline          nan   0.57627119     102/177    
qwen_v2           0.70832985   0.80790960     143/177    
chatgpt_v1        0.74333246   0.73446328     130/177    
gpt5_v3           0.73301657   0.70621469     125/177    
deepseek_v1       0.65300884   0.76271186     135/177    
urdu_v2           0.60128077   0.70621469     125/177    
david_v2          0.60521126   0.68361582     121/177    
qwen_v1           0.63082859   0.64406780     114/177    
gemini            0.63467552   0.61016949     108/177    
urdu_v1           0.43012813   0.69491525     123/177    


/tmp/ipykernel_274773/1580405741.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_value = spearmanr(pred_list, gold_list)


# Ensemble

## Mean

In [67]:
# Simple Mean Ensemble: y_pred = (1/N) * sum(y_i)
y_pred_mean = X_test.mean(axis=1).values

# Evaluate float version
scores_mean_float = evaluate_predictions_array(y_pred_mean, test_gold_labels)
print(f"Mean (float) - Spearman: {scores_mean_float['spearman']:.4%}, Accuracy: {scores_mean_float['accuracy']:.4%}")

# Evaluate integer version
y_pred_mean_int = y_pred_mean.round().clip(1, 5).astype(int)
scores_mean_int = evaluate_predictions_array(y_pred_mean_int, test_gold_labels)
print(f"Mean (int)   - Spearman: {scores_mean_int['spearman']:.4%}, Accuracy: {scores_mean_int['accuracy']:.4%}")

Mean (float) - Spearman: 84.6561%, Accuracy: 92.0904%
Mean (int)   - Spearman: 79.0823%, Accuracy: 82.4859%


## Linear Stacking (Ridge Regression)

In [68]:
def train_eval_ridge(X_train, y_train, X_test, test_gold_labels, alpha=1.0):
    """
    Train and evaluate Ridge Regression ensemble.

    Args:
        X_train: Training features
        y_train: Training labels
        X_test: Test features
        test_gold_labels: Test gold labels (list of 5 ratings each)
        alpha: Ridge regularization parameter

    Returns:
        scores_float, scores_int, ridge_model
    """
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)

    # Predict on test set
    y_pred_ridge = ridge.predict(X_test)

    # Evaluate float version
    scores_float = evaluate_predictions_array(y_pred_ridge, test_gold_labels)

    # Evaluate integer version
    y_pred_int = y_pred_ridge.round().clip(1, 5).astype(int)
    scores_int = evaluate_predictions_array(y_pred_int, test_gold_labels)

    return scores_float, scores_int, ridge


def grid_search_ridge(X_train, y_train, X_test, test_gold_labels, alphas=[0.1, 1.0, 10.0], verbose=True):
    """
    Grid search over Ridge alpha values.

    Args:
        X_train, y_train, X_test, test_gold_labels: Data splits
        alphas: List of alpha values to try
        verbose: Whether to print detailed progress

    Returns:
        best_alpha, best_scores_float, best_scores_int, best_model
    """
    best_avg_score = 0
    best_alpha = None
    best_scores_float = None
    best_scores_int = None
    best_model = None

    for alpha in alphas:
        scores_float, scores_int, model = train_eval_ridge(
            X_train, y_train, X_test, test_gold_labels, alpha=alpha
        )

        avg_score = (scores_float['spearman'] + scores_float['accuracy']) / 2

        if verbose:
            print(f"   α={alpha:5.1f} → Spearman: {scores_float['spearman']:.2%}, "
                  f"Accuracy: {scores_float['accuracy']:.2%}, "
                  f"Avg: {avg_score:.2%}")

        if avg_score > best_avg_score:
            best_avg_score = avg_score
            best_alpha = alpha
            best_scores_float = scores_float
            best_scores_int = scores_int
            best_model = model

    if verbose:
        print(f"\n   ✓ Best: α={best_alpha} with average score {best_avg_score:.2%}")

    return best_alpha, best_scores_float, best_scores_int, best_model

In [69]:
def test_model_combination(combination_name, model_names, train_df, test_df, test_gold_labels, alphas=[0.1, 1.0, 10.0]):
    """
    Test a specific model combination with Ridge regression.

    Args:
        combination_name: Name of the combination (for display)
        model_names: List of model column names to use
        train_df, test_df: Data splits
        test_gold_labels: Gold labels for test set
        alphas: Alpha values to try

    Returns:
        best_alpha, best_scores_float, best_scores_int, best_model, X_train, X_test
    """
    # Prepare data
    X_train = train_df[model_names]
    y_train = train_df['gold_avg']
    X_test = test_df[model_names]
    y_test = test_df['gold_avg']

    # Run grid search
    best_alpha, best_scores_float, best_scores_int, best_model = grid_search_ridge(
        X_train, y_train, X_test, test_gold_labels, alphas=alphas, verbose=False
    )

    return best_alpha, best_scores_float, best_scores_int, best_model, X_train, X_test

def compare_all_combinations(train_df, test_df, test_gold_labels, model_combinations, alphas=[0.1, 1.0, 10.0]):
    """
    Compare all model combinations and summarize results.

    Args:
        train_df, test_df: Data splits
        test_gold_labels: Gold labels for test set
        model_combinations: Dict of {name: [model_list]}
        alphas: Alpha values to try

    Returns:
        results_summary: DataFrame with comparison results
    """
    results = []

    for combo_name, model_names in model_combinations.items():
        best_alpha, scores_float, scores_int, model, X_train, X_test = test_model_combination(
            combo_name, model_names, train_df, test_df, test_gold_labels, alphas
        )

        results.append({
            'combination': combo_name,
            'num_models': len(model_names),
            'models': ', '.join(model_names),
            'best_alpha': best_alpha,
            'spearman': scores_float['spearman'],
            'accuracy': scores_float['accuracy'],
            'avg_score': (scores_float['spearman'] + scores_float['accuracy']) / 2,
        })

    # Create summary DataFrame sorted by average score
    summary_df = pd.DataFrame(results)
    summary_df = summary_df.sort_values('avg_score', ascending=False)
    
    # Display table ranked by average
    print("\n📋 Results (Ranked by Average Score):")
    display_df = summary_df[['combination', 'num_models', 'avg_score', 'spearman', 'accuracy', 'best_alpha']].copy()
    display_df.columns = ['Combination', '#Models', 'Avg Score', 'Spearman', 'Accuracy', 'Best α']
    
    # Format percentages
    for col in ['Avg Score', 'Spearman', 'Accuracy']:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.2%}")
    
    display(display_df.reset_index(drop=True))

    return summary_df

In [ ]:
combinations_to_compare = {}

# for size in range(2, len(all_systems) + 1):  # Start from 2 to skip single models
for combo in [['chatgpt_v1','david_v2','deepseek_v1','gemini','gpt5_v1','gpt5_v3','gpt5_v4','korean_v2','qwen_v1','urdu_v2']]:# combinations(all_systems,):
        # Use shortened names for cleaner output
        combo_name = '+'.join(combo)
        combinations_to_compare[combo_name] = list(combo)

summary = compare_all_combinations(
    train_df,
    test_df,
    test_gold_labels,
    combinations_to_compare,
    alphas=[10.0]
)


📋 Results (Ranked by Average Score):


,Combination,#Models,Avg Score,Spearman,Accuracy,Best α
0,chatgpt_v1+david_v2+deepseek_v1+gemini+gpt5_v1...,10,89.08%,84.37%,93.79%,5


In [47]:
print("\n✅ Ensemble evaluation complete.")
print("\nSummary of all model combinations evaluated:")
# row 1
print(summary.to_string(index=False))


✅ Ensemble evaluation complete.

Summary of all model combinations evaluated:
                               combination  num_models                                         models  best_alpha  spearman  accuracy  avg_score
david_v2+gpt5_v1+korean_v2+qwen_v1+urdu_v2           5 david_v2, gpt5_v1, korean_v2, qwen_v1, urdu_v2        0.01  0.846767  0.932203   0.889485


In [48]:
# Example 2: Compare all predefined combinations
# Uncomment and run to compare all combinations:

# summary = compare_all_combinations(
#     train_df,
#     test_df,
#     test_gold_labels,
#     MODEL_COMBINATIONS
# )

In [49]:
# Example 1: Test a single combination
# Uncomment and run to test just one combination:

# test_model_combination(
#     'best_5',
#     MODEL_COMBINATIONS['best_5'],
#     train_df,
#     test_df,
#     test_gold_labels
# )

In [50]:
# # Inspect Ridge Weights
# coef = ridge.coef_
# for name, w in zip(X_train.columns, coef):
#     print(f"{name:20s} {w:+.4f}")

In [51]:
# david_v1+david_v2+korean_v1+korean_v2+qwen_v1+qwen_v2+urdu_v2
best_models = [ 'david_v1', 'david_v2', 'korean_v1', 'korean_v2', 'qwen_v1', 'qwen_v2', 'urdu_v2' ]
X_train, y_train, X_test, y_test = prepare_data_splits(
    train_df,
    test_df,
    best_models
)

## XGBoost Regressor

In [52]:
def train_eval_xgb(
    X_train,
    y_train,
    X_test,
    test_gold_labels,
    *,
    n_estimators=500,
    max_depth=2,
    learning_rate=0.05,
    min_child_weight=10,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1.0,     # L1
    reg_lambda=10.0,   # L2
    random_state=42,
):
    xgb = XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        objective="reg:squarederror",
        random_state=random_state,
        n_jobs=-1,
    )

    # Fit with early stopping (VERY IMPORTANT)
    xgb.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train)],
        verbose=False,
    )

    # Predict
    y_pred = xgb.predict(X_test)

    # Float eval
    scores_float = evaluate_predictions_array(y_pred, test_gold_labels)

    # Int eval
    y_pred_int = y_pred.round().clip(1, 5).astype(int)
    scores_int = evaluate_predictions_array(y_pred_int, test_gold_labels)

    return scores_float, scores_int


In [114]:
def compress_as_needed(list_o_thing, factor):
    

    is_array = isinstance(list_o_thing, np.ndarray)

    
    ans = [] if not is_array else np.empty_like(list_o_thing, dtype=float)

    for i, guy in enumerate(list_o_thing):
        if guy < 2:
            new_guy = 2 - (2 - guy) * factor
            assert new_guy <= 2 and new_guy >= guy
        elif guy > 4:
            new_guy = 4 + (guy - 4) * factor
            assert new_guy >= 4 and new_guy <= guy
        else:
            new_guy = guy  # untouched inside [2, 4]

        if is_array:
            ans[i] = new_guy
        else:
            ans.append(float(new_guy))
   
    
    return ans

In [122]:
def is_same_with_less_scaling(a, b):
    print(a)
    print(b)
    assert False

def any_self_with_less_scaling_is_worse(r, p):
    
    for guy in r:
        if guy != p and is_same_with_less_scaling(guy, p):
            assert guy['avg_score'] <= p['avg_score']
    return True

In [ ]:
from itertools import product
from tqdm import tqdm
import math

def grid_search_xgboost(test_gold_labels, param_grid=None):
    """
    Grid search over XGBoost hyperparameters.
    
    Args:
        X_train, y_train: Training data
        X_test, test_gold_labels: Test data
        param_grid: Dictionary of parameters to search. If None, uses default grid.
    
    Returns:
        best_params, best_scores_float, best_scores_int, best_model, results_df
    """
    from xgboost import XGBRegressor
    
    # Default parameter grid
    if param_grid is None:
        param_grid = {
            'n_estimators': [100, 200, 500],
            'max_depth': [1, 2, 3],
            'learning_rate': [0.01, 0.05, 0.1],
            'min_child_weight': [10, 50, 100],
            'subsample': [0.7, 0.8, 0.9],
            'colsample_bytree': [0.7, 0.8, 0.9],
            'reg_alpha': [0.1, 1.0, 10.0],
            'reg_lambda': [1.0, 10.0, 50.0],
        }
    
    results = []
    best_avg_score = 0
    best_params = None
    best_scores_float = None
    best_scores_int = None
    best_model = None
    
    # Generate all combinations
    import itertools
    keys = param_grid.keys()
    values = param_grid.values()
    
    total_combinations = 1
    for v in values:
        total_combinations *= len(v)
    
    
    total = math.prod(len(v) for v in values)

    
    for i, combination in enumerate(
    tqdm(product(*values), total=total),
    start=1
    ):
        params = dict(zip(keys, combination))
        
        model_names = params['model_names']
        
        X_train_curr, y_train_curr, X_test_curr, y_test_curr = prepare_data_splits(train_df, test_df, model_names)
        
        # Train XGBoost with these parameters
        xgb = XGBRegressor(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            min_child_weight=params['min_child_weight'],
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            reg_alpha=params['reg_alpha'],
            reg_lambda=params['reg_lambda'],
            objective="reg:squarederror",
            random_state=0,
            n_jobs=-1,
        )
        
        xgb.fit(X_train_curr, y_train_curr, verbose=False)
        
        
        
        # Predict and evaluate
        y_pred = compress_as_needed(xgb.predict(X_test_curr), params['rescale'])
        scores_float = evaluate_predictions_array(y_pred, test_gold_labels)
        
        y_pred_int = y_pred.round().clip(1, 5).astype(int)
        scores_int = evaluate_predictions_array(y_pred_int, test_gold_labels)
        
        avg_score = (scores_float['spearman'] + scores_float['accuracy']) / 2
        
        
        # Store results
        results.append({
            **params,
            'spearman': scores_float['spearman'],
            'accuracy': scores_float['accuracy'],
            'avg_score': avg_score,
        })
        
        # Track best
        if avg_score > best_avg_score:
            best_avg_score = avg_score
            best_params = params
            best_scores_float = scores_float
            best_scores_int = scores_int
            best_model = xgb
        
        # Progress update every 10%
        if i % max(1, total_combinations // 1000) == 0:
            # print(f"  Progress: {i}/{total_combinations} ({i/total_combinations*100:.0f}%)")
            pass
        
        for curr_param in results:
            any_self_with_less_scaling_is_worse(results, curr_param)
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    
    results_df = results_df.sort_values('avg_score', ascending=False)
    
    print(f"\n✅ Best parameters found:")
    for key, val in best_params.items():
        print(f"   {key:20s} = {val}")
    print(f"\n   Avg Score: {best_avg_score:.2%}")
    print(f"   Spearman:  {best_scores_float['spearman']:.2%}")
    print(f"   Accuracy:  {best_scores_float['accuracy']:.2%}")
    
    return best_params, best_scores_float, best_scores_int, best_model, results_df


In [124]:
KOREAN_PARTS = [
    'korean_ambiguity',
    'korean_correct',
    'korean_neither',
    'korean_preference',
    'korean_rating',
]

def only_combined_korean(s):
    return (not any(p in s for p in KOREAN_PARTS)) and ('korean_v' in s)

def all_components_korean(s):
    return (all(p in s for p in KOREAN_PARTS)) and ('korean_v' not in s)


def this_is_a_good_combo(c):
    
    s = str(c)
 
    for required in ['david', 'korean', 'urdu', 'v2']:
        if required not in str(c):
            return False
    if 'qw' not in str(c) and 'gpt5' not in str(c) and 'deep' not in str(c):
        return False
    if 'david_v1' in str(c) or 'david_v3' in str(c):
        return False
    if 'urdu_v1' in str(c) or  'urdu_v3' in str(c):
        return False
    if 'korean_v2' in str(c):
        return False      
    if only_combined_korean(s) == all_components_korean(s):
        return False
    

    return True



In [125]:
combinations_to_compare = []
for size in range(2, len(all_systems) + 2):  # Start from 2 to skip single models
    for combo in combinations(all_systems, size):
        # Use shortened names for cleaner output
        if this_is_a_good_combo(combo):
            combo_name = '+'.join(combo)
            combinations_to_compare.append(list(combo))



with open('combstocomp.out', 'w', encoding='utf-8') as fff:
    for comb_to_comp in combinations_to_compare:
        fff.write(' '.join(comb_to_comp) + '\n')



custom_grid = {
    'n_estimators': [500],
    'max_depth': [1],
    'rescale': [0, 0.01, 0.25, 1],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_weight': [30, 50],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'reg_alpha': [1.0, 10.0],
    'reg_lambda': [1.0, 10.0],
    'model_names': combinations_to_compare
}

best_params, scores_float, scores_int, best_model, results_df = grid_search_xgboost(
    test_gold_labels, custom_grid
)

# View top 10 parameter combinations
print("\n📋 Top 10 Parameter Combinations:")
display(results_df.head(10))

  0%|          | 1/11520 [00:00<20:16,  9.47it/s]

{'n_estimators': 500, 'max_depth': 1, 'rescale': 0, 'learning_rate': 0.01, 'min_child_weight': 30, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 1.0, 'reg_lambda': 1.0, 'model_names': ['david_v2', 'deepseek_v1', 'korean_v1', 'urdu_v2'], 'spearman': np.float64(0.8295323786993674), 'accuracy': 0.8305084745762712, 'avg_score': np.float64(0.8300204266378193)}
{'n_estimators': 500, 'max_depth': 1, 'rescale': 0, 'learning_rate': 0.01, 'min_child_weight': 30, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 1.0, 'reg_lambda': 1.0, 'model_names': ['david_v2', 'gpt5_v1', 'korean_v1', 'urdu_v2']}


AssertionError: 

In [82]:


a = 0
with open("KEEPTHISINMINDOK5.out", 'w', encoding='utf-8') as in_f:
    for index, row in results_df.iterrows():
        if a < 10000:
            in_f.write(str(row['model_names']) + '\n')
            in_f.write(str(row) + '\n\n\n\n')
        a += 1

In [105]:
# XGBoost with competition-validated settings
xgb = XGBRegressor(
    n_estimators=500,          # matched to lr=0.1
    max_depth=1,               # CRITICAL: best Spearman
    learning_rate=0.05,
    min_child_weight=30,       # strong regularization
    subsample=0.7,
    colsample_bytree=0.8,
    reg_alpha=10.0,             # L1 regularization
    reg_lambda=10.0,           # L2 regularization
    objective="reg:squarederror",
    random_state=0,
    n_jobs=-1,
)

# Train on training set
xgb.fit(X_train, y_train)

# Predict on test set
y_pred_xgb = xgb.predict(X_test)

# Evaluate float version
scores_xgb_float = evaluate_predictions_array(y_pred_xgb, test_gold_labels)
print(
    f"XGBoost depth=1 (float) - "
    f"Average: {(scores_xgb_float['spearman'] + scores_xgb_float['accuracy'])/2:.4%}, "
    f"Spearman: {scores_xgb_float['spearman']:.4%}, "
    f"Accuracy: {scores_xgb_float['accuracy']:.4%}"
)

# Evaluate integer version
y_pred_xgb_int = y_pred_xgb.round().clip(1, 5).astype(int)
scores_xgb_int = evaluate_predictions_array(y_pred_xgb_int, test_gold_labels)
print(
    f"XGBoost depth=1 (int)   - "
    f"Average: {(scores_xgb_int['spearman'] + scores_xgb_int['accuracy'])/2:.4%}, "
    f"Spearman: {scores_xgb_int['spearman']:.4%}, "
    f"Accuracy: {scores_xgb_int['accuracy']:.4%}"
)


XGBoost depth=1 (float) - Average: 88.8299%, Spearman: 83.3095%, Accuracy: 94.3503%
XGBoost depth=1 (int)   - Average: 80.2304%, Spearman: 77.4100%, Accuracy: 83.0508%


In [150]:
# Inspect XGBoost Feature Importances
importance = xgb.feature_importances_
print("XGBoost Feature Importances:")
for name, imp in zip(X_train.columns, importance):
    print(f"{name:20s} {imp:.4f}")
    
# Visualize as percentages
print("\nNormalized Contributions (%):")
total_importance = importance.sum()
for name, imp in zip(X_train.columns, importance):
    print(f"{name:20s} {imp/total_importance:>7.2%}")

XGBoost Feature Importances:
david_v2             0.0884
korean_v1            0.1413
korean_v2            0.7069
urdu_v2              0.0493
qwen_v2              0.0140

Normalized Contributions (%):
david_v2               8.84%
korean_v1             14.13%
korean_v2             70.69%
urdu_v2                4.93%
qwen_v2                1.40%


## XGBoost Ranker

In [141]:
# Helper function to scale predictions back to 1-5 range
def scale_to_range(arr, low=1, upp=5):
    """Scale array values to be within the specified range [low, upp]."""
    arr_min, arr_max = arr.min(), arr.max()
    if arr_max == arr_min:
        return np.full_like(arr, (low + upp) / 2)
    arr_scaled = (arr - arr_min) / (arr_max - arr_min) * (upp - low) + low
    return arr_scaled

In [142]:
def train_eval_xgb_ranker(
    X_train,
    y_train,
    X_test,
    test_gold_labels,
    group_size=6,
    *,
    n_estimators=500,
    max_depth=2,
    learning_rate=0.05,
    min_child_weight=10,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1.0,     # L1
    reg_lambda=10.0,   # L2
    random_state=42,
):
    """
    Train and evaluate XGBRanker.
    
    Args:
        group_size: Number of samples per group for ranking
                   (e.g., 6 means every 6 consecutive samples form a ranking group)
    """
    # Create group information for ranking
    # Each group contains group_size samples to be ranked together
    num_groups_train = len(X_train) // group_size
    group_train = [group_size] * num_groups_train
    
    # Initialize XGBRanker
    xgb_ranker = XGBRanker(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        objective="rank:pairwise",  # Pairwise ranking objective
        random_state=random_state,
        n_jobs=-1,
    )

    # Fit with group information
    xgb_ranker.fit(
        X_train[:num_groups_train * group_size],  # Use only complete groups
        y_train[:num_groups_train * group_size],
        group=group_train,
        verbose=False,
    )

    # Predict - returns ranking scores
    y_pred_scores = xgb_ranker.predict(X_test)
    
    # Scale scores back to 1-5 range
    y_pred = scale_to_range(y_pred_scores, low=1, upp=5)

    # Float eval
    scores_float = evaluate_predictions_array(y_pred, test_gold_labels)

    # Int eval
    y_pred_int = y_pred.round().clip(1, 5).astype(int)
    scores_int = evaluate_predictions_array(y_pred_int, test_gold_labels)

    return scores_float, scores_int, xgb_ranker

In [143]:
# Grid search over XGBoost Ranker hyperparameters
print("=" * 80)
print("XGBoost Ranker Hyperparameter Search")
print("=" * 80)

configs_ranker = [
    dict(learning_rate=0.1,  n_estimators=100, group_size=6),
    dict(learning_rate=0.05, n_estimators=200, group_size=6),
    dict(learning_rate=0.02, n_estimators=400, group_size=6),
]

for depth in [1, 2, 3]:
    print(f"\nDepth={depth}")
    print("-" * 80)
    for cfg in configs_ranker:
        scores_float, scores_int, _ = train_eval_xgb_ranker(
            X_train,
            y_train,
            X_test,
            test_gold_labels,
            max_depth=depth,
            **cfg
        )

        print(
            f"  lr={cfg['learning_rate']:4.2f} n={cfg['n_estimators']:3d} group={cfg['group_size']} | "
            f"Spearman={scores_float['spearman']:.4%} Acc={scores_float['accuracy']:.4%}"
        )

XGBoost Ranker Hyperparameter Search

Depth=1
--------------------------------------------------------------------------------
  lr=0.10 n=100 group=6 | Spearman=83.3431% Acc=92.0904%
  lr=0.05 n=200 group=6 | Spearman=83.5367% Acc=93.7853%
  lr=0.02 n=400 group=6 | Spearman=83.7295% Acc=92.6554%

Depth=2
--------------------------------------------------------------------------------
  lr=0.10 n=100 group=6 | Spearman=82.9971% Acc=88.7006%
  lr=0.05 n=200 group=6 | Spearman=82.9917% Acc=90.9605%
  lr=0.02 n=400 group=6 | Spearman=83.0810% Acc=89.8305%

Depth=3
--------------------------------------------------------------------------------
  lr=0.10 n=100 group=6 | Spearman=82.6533% Acc=90.3955%
  lr=0.05 n=200 group=6 | Spearman=83.0643% Acc=89.2655%
  lr=0.02 n=400 group=6 | Spearman=82.8186% Acc=89.2655%


In [144]:
# Train final XGBoost Ranker with best settings
print("=" * 80)
print("Final XGBoost Ranker Model")
print("=" * 80)

xgb_ranker = XGBRanker(
    n_estimators=400,
    max_depth=1,
    learning_rate=0.05,
    min_child_weight=10,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1.0,
    reg_lambda=10.0,
    objective="rank:pairwise",
    n_jobs=-1,
)

# Create groups (samples per group)
group_size = 6
num_groups_train = len(X_train) // group_size
group_train = [group_size] * num_groups_train

# Train on complete groups only
X_train_grouped = X_train[:num_groups_train * group_size]
y_train_grouped = y_train[:num_groups_train * group_size]

xgb_ranker.fit(X_train_grouped, y_train_grouped, group=group_train)

# Predict on test set
y_pred_ranker_scores = xgb_ranker.predict(X_test)
y_pred_ranker = scale_to_range(y_pred_ranker_scores, low=1, upp=5)

# Evaluate float version
scores_ranker_float = evaluate_predictions_array(y_pred_ranker, test_gold_labels)
print(f"XGBoost Ranker (float) - Average: {(scores_ranker_float['spearman'] + scores_ranker_float['accuracy'])/2:.4%}, "
      f"Spearman: {scores_ranker_float['spearman']:.4%}, Accuracy: {scores_ranker_float['accuracy']:.4%}")

# Evaluate integer version
y_pred_ranker_int = y_pred_ranker.round().clip(1, 5).astype(int)
scores_ranker_int = evaluate_predictions_array(y_pred_ranker_int, test_gold_labels)
print(f"XGBoost Ranker (int)   - Average: {(scores_ranker_int['spearman'] + scores_ranker_int['accuracy'])/2:.4%}, "
      f"Spearman: {scores_ranker_int['spearman']:.4%}, Accuracy: {scores_ranker_int['accuracy']:.4%}")

Final XGBoost Ranker Model
XGBoost Ranker (float) - Average: 88.6203%, Spearman: 83.4553%, Accuracy: 93.7853%
XGBoost Ranker (int)   - Average: 83.7981%, Spearman: 81.1555%, Accuracy: 86.4407%


In [145]:
# Inspect XGBoost Ranker Feature Importances
importance_ranker = xgb_ranker.feature_importances_
print("XGBoost Ranker Feature Importances:")
for name, imp in zip(X_train.columns, importance_ranker):
    print(f"{name:20s} {imp:.4f}")
    
# Visualize as percentages
print("\nNormalized Contributions (%):")
total_importance_ranker = importance_ranker.sum()
for name, imp in zip(X_train.columns, importance_ranker):
    print(f"{name:20s} {imp/total_importance_ranker:>7.2%}")

XGBoost Ranker Feature Importances:
david_v2             0.1766
korean_v1            0.2209
korean_v2            0.2235
urdu_v2              0.1518
qwen_v2              0.2271

Normalized Contributions (%):
david_v2              17.66%
korean_v1             22.09%
korean_v2             22.35%
urdu_v2               15.18%
qwen_v2               22.71%
